# Projet : CryptoBot avec Binance (OPA)

## Phase 2 : Organisation des Données

### Objectif du Document:

Ce doc a pour but de présenter les différentes étapes pour : 
- Réupérer une donnée Historique, 
- Alimenter la Base Mongo, 
- Alimenter la table SQL Histo

### Step 1:  Initialisation des Modules

In [1]:
from Binance.Data import Binance_Histo as Histo
from Binance.Data import Binance_Live as live
from Binance.Dao import Drivers_MongoDB as DAO_MB
from Binance.Dao import Drivers_SQlite as DAO_SQL
from Binance.Utils import Utilitaire as util
from Binance.Utils import Technical_Analyst as util_TA

import pandas as pd

Symbol = 'BTCUSDT'

### Step 2: Chargement Fichiers Historique 

In [11]:
L = list()

X = Histo.Binance_Histo([Symbol], ['15m'], Frequence= 'M')
X.get_ListeFichier()
X.TelechargeFichier()

for Monfichier in X.L_Fichier:
    L.append(Monfichier['Nom'])

print('ci dessous la liste des fichiers à charger')
print(L[0:2])


ci dessous la liste des fichiers à charger
['BTCUSDT-15m-2022-12.csv']


### Step 3: Connexion à la base Mongo DB

In [12]:
# Connexion à la base avec les fichiers à charger
Y = DAO_MB.Drivers_MongoDB(L)
Y.ChargeFichiers()

In [2]:
# Connexion à la base sans fichiers à charger
Y = DAO_MB.Drivers_MongoDB()

In [9]:
# Suppression des collections si besoin
Y.DeleteCollection([Symbol])

In [3]:
print('Liste des Collection :',  Y.get_AllCollection())
print('Liste des Fichiers Chargés de la 1ere collection :', Y.get_ListeFichier(Y.get_AllCollection()[0]))
X.SupprimeFichier()

Liste des Collection : ['BTCUSDT']
Liste des Fichiers Chargés de la 1ere collection : ['BTCUSDT-15m-2022-12.csv']


NameError: name 'X' is not defined

### Step 4:Connexion à la base SQLite

In [5]:
#Connexion à la base SQLite

PathDatabase = '/home/arnold/ENV_VIRTUEL/ATU_FORMATION/REP_DEV/Projet_OPA/DataBase/SQLite/test.db'
SQL = DAO_SQL.Drivers_SQLite(PathDatabase)


In [3]:
#Reinitialisation de la DataBase (Si besoin)

PathCreateTable = '/home/arnold/ENV_VIRTUEL/ATU_FORMATION/REP_DEV/Projet_OPA/Binance/Dao/Create_DBSQLITE_OPA.sql'
SQL.Re_InitDB(PathCreateTable)

#### Step 4.1: Alimentation Table Dim Temps

In [26]:
#Récupération des données temps depuis la Base Mongo

Liste_Temps = list(Y.DBMongo[Symbol].find({}, {'Detail.Close_time' : 1, 'Detail.Open_time': 1}))

In [27]:
# Formatage du Résultat dns un DataFrame

R = list()
for i in Liste_Temps :
    for y in i['Detail']:
        if y['Open_time'] not in R:
            R.append(y['Open_time'])
        
        if y['Close_time'] not in R:
            R.append(y['Close_time'])
            
DimTemps = pd.DataFrame(R, columns = ['ID_TEMPS'] , dtype='int')

DimTemps.head()

,ID_TEMPS
0,1672186500000
1,1672187399999
2,1672187400000
3,1672188299999
4,1672188300000


In [28]:
# Mise à jour du DataFrame au format de la table DimTemps

DimTemps['SECONDES'] = DimTemps['ID_TEMPS'].apply(util.Convertir_Timestamp, formatDate=('ss'))
DimTemps['MINUTES'] = DimTemps['ID_TEMPS'].apply(util.Convertir_Timestamp, formatDate=('mm'))
DimTemps['HEURE'] = DimTemps['ID_TEMPS'].apply(util.Convertir_Timestamp, formatDate=('HH'))
DimTemps['JOUR'] = DimTemps['ID_TEMPS'].apply(util.Convertir_Timestamp, formatDate=('DD'))
DimTemps['MOIS'] = DimTemps['ID_TEMPS'].apply(util.Convertir_Timestamp, formatDate=('MM'))
DimTemps['ANNEE'] = DimTemps['ID_TEMPS'].apply(util.Convertir_Timestamp, formatDate=('YYYY'))
    
DimTemps.head(5)

,ID_TEMPS,SECONDES,MINUTES,HEURE,JOUR,MOIS,ANNEE
0,1672186500000,0,15,1,28,12,2022
1,1672187399999,59,29,1,28,12,2022
2,1672187400000,0,30,1,28,12,2022
3,1672188299999,59,44,1,28,12,2022
4,1672188300000,0,45,1,28,12,2022


In [29]:
# Alimentation Table Dim_Temps

SQL.Alim_DimTemps(DimTemps)
res = SQL.Select('select * from DIM_TEMPS limit 5;')
print(res)


[(1672186500000, '0', '15', '1', '28', '12', '2022', '2023-01-27'), (1672187399999, '59', '29', '1', '28', '12', '2022', '2023-01-27'), (1672187400000, '0', '30', '1', '28', '12', '2022', '2023-01-27'), (1672188299999, '59', '44', '1', '28', '12', '2022', '2023-01-27'), (1672188300000, '0', '45', '1', '28', '12', '2022', '2023-01-27')]


#### Step 4.2: Alimentation Table Dim Symbol

In [30]:
#Récupération des données symbole depuis la Base Mongo

Liste_Symbole = list(Y.DBMongo[Symbol].find({}, {"Symbol" : 1, "Intervalle" : 1, "_id" : 0}))

R2 = list()
for i in Liste_Symbole:
    a = {"NOM_SYMBOL" : i['Symbol'], "INTERVALLE" : i['Intervalle']} 
    if a not in R2:
        R2.append(a)
        
DimSymbol = pd.DataFrame(R2)

In [31]:
# On compléte le df Symbol

DLive = live.Binance_Live()
Info_symbol = DLive.exchange_info(Symbol)


i = {'NOM_SYMBOL' : Info_symbol['symbols'][0]['symbol'],
                              'BaseAsset' : Info_symbol['symbols'][0]['baseAsset'],
                              'QuoteAsset' : Info_symbol['symbols'][0]['quoteAsset']}
df = pd.DataFrame([i ])
DimSymbol = DimSymbol.merge(df, on = 'NOM_SYMBOL' )



In [32]:
SQL.Alim_DimSymbol(DimSymbol)
res = SQL.Select('select * from DIM_SYMBOL limit 5;')
print(res)
            

[(1, '1INCHBUSD', '15m', '1INCH', 'BUSD', '2023-01-27'), (2, '1INCHBUSD', '30m', '1INCH', 'BUSD', '2023-01-27')]


#### Step 4.3: Alimentation Table Fait Situation Histo

In [11]:
# Récupération des données Histo de la base Mongo
L = list()

for doc in Y.get_AllDocuments(Symbol):
    for detail in doc['Detail']:
        L.append({'ID_TEMPS' :detail['Close_time'], 
                  'NOM_SYMBOL' : doc['Symbol'], 
                  'INTERVALLE' : doc['Intervalle'], 
                  'VALEUR_COURS' : detail['Close'] ,
                  'IND_QUOTEVOLUME' : detail['Quote_asset_volume'] 
                 })

FaiCoursHisto = pd.DataFrame(L)
FaiCoursHisto['ID_TEMPS'] = FaiCoursHisto['ID_TEMPS'].astype(int)
FaiCoursHisto['IND_QUOTEVOLUME'] = FaiCoursHisto['IND_QUOTEVOLUME'].astype(int)

FaiCoursHisto.head()


,ID_TEMPS,NOM_SYMBOL,INTERVALLE,VALEUR_COURS,IND_QUOTEVOLUME
0,1669854599999,BTCUSDT,15m,17202.17,90925762
1,1669855499999,BTCUSDT,15m,17155.80,43014195
2,1669856399999,BTCUSDT,15m,17161.55,43985336
3,1669857299999,BTCUSDT,15m,17145.88,37258575
4,1669858199999,BTCUSDT,15m,17123.66,48736734


In [12]:
#Recherche Id Symbol à partir de la base SQL
L = list()

res = SQL.Select('select ID_SYMBOL,NOM_SYMBOL,INTERVALLE  from DIM_SYMBOL;')

for i in res:
    (a,b,c) = i
    L.append({'ID_SYMBOL' : a, 'NOM_SYMBOL' : b, 'INTERVALLE' : c})
    
df = pd.DataFrame(L)

FaiCoursHisto = FaiCoursHisto.merge(df, how = 'inner')

FaiCoursHisto.head()

,ID_TEMPS,NOM_SYMBOL,INTERVALLE,VALEUR_COURS,IND_QUOTEVOLUME,ID_SYMBOL
0,1669854599999,BTCUSDT,15m,17202.17,90925762,1
1,1669855499999,BTCUSDT,15m,17155.80,43014195,1
2,1669856399999,BTCUSDT,15m,17161.55,43985336,1
3,1669857299999,BTCUSDT,15m,17145.88,37258575,1
4,1669858199999,BTCUSDT,15m,17123.66,48736734,1


In [13]:
# Calcul des Indicateurs
FaiCoursHisto['IND_SMA_20'] = util_TA.Calculer_SMA(FaiCoursHisto['VALEUR_COURS'], 20)
FaiCoursHisto['IND_SMA_30'] = util_TA.Calculer_SMA(FaiCoursHisto['VALEUR_COURS'], 30)
FaiCoursHisto['IND_CHANGEPERCENT'] = util_TA.Calculer_Change_Percent(FaiCoursHisto['VALEUR_COURS'])
FaiCoursHisto['IND_STOCH_RSI'] = util_TA.Calculer_RSI_Stochastique(FaiCoursHisto['VALEUR_COURS'])
FaiCoursHisto['IND_RSI'] =  util_TA.Calculer_RSI(FaiCoursHisto['VALEUR_COURS'])
FaiCoursHisto['IND_TRIX'] = util_TA.calculate_trix(FaiCoursHisto['VALEUR_COURS'])

FaiCoursHisto = FaiCoursHisto[['ID_TEMPS',  'ID_SYMBOL','VALEUR_COURS', 'IND_SMA_20', 'IND_SMA_30', 'IND_QUOTEVOLUME', 'IND_CHANGEPERCENT', 'IND_STOCH_RSI', 'IND_RSI', 'IND_TRIX']]

FaiCoursHisto.head(5)

,ID_TEMPS,ID_SYMBOL,VALEUR_COURS,IND_SMA_20,IND_SMA_30,IND_QUOTEVOLUME,IND_CHANGEPERCENT,IND_STOCH_RSI,IND_RSI,IND_TRIX
0,1669854599999,1,17202.17,NaN,NaN,90925762,NaN,NaN,NaN,NaN
1,1669855499999,1,17155.80,NaN,NaN,43014195,-0.270288,NaN,NaN,NaN
2,1669856399999,1,17161.55,NaN,NaN,43985336,0.033505,NaN,NaN,NaN
3,1669857299999,1,17145.88,NaN,NaN,37258575,-0.091392,NaN,NaN,NaN
4,1669858199999,1,17123.66,NaN,NaN,48736734,-0.129762,NaN,NaN,NaN


In [ ]:
# Alimentation table SQL

SQL.Alim_FaitSituation_Histo(FaiCoursHisto)
res = SQL.Select('select * from FAIT_SIT_COURS_HIST limit 5;')
print(res)
            